<a href="https://colab.research.google.com/github/Hybern8/EntLife/blob/version2/Technical_dept_transfomer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [15]:
%%R


install.packages("openxlsx")

# Load in the libraries

library(dplyr) # for data manipulation
library(tidyr) # for data manipulation
library(openxlsx) # for date data type

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
folder_path = '/content/drive/Othercomputers/Decipher/Documents/Quarterhill/TechRep'

In [18]:
%%R -i folder_path
setwd(folder_path)

# Verify the working directory
getwd()

[1] "/content/drive/Othercomputers/Decipher/Documents/Quarterhill/TechRep"


In [19]:
%%R
# Read in data for column names
col_names <- read.xlsx("Ref_file/col_names.xlsx", colNames = T)

# Read in data & split columns
Raw <- read.csv('dataset/Raw.csv',stringsAsFactors = F)
data <- Raw %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.1`,
           into = c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.2`,
           into = c("A1", "B1", "C1", "D1", "E1", "F1", "G1", "H1", "I1", "J1", "K1", "L1"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.3`,
           into = c("A2", "B2", "C2", "D2", "E2", "F2", "G2", "H2", "I2", "J2"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.4`,
           into = c("A3", "B3", "C3", "D3", "E3", "F3", "G3", "H3", "I3"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.5`,
           into = c("A4", "B4", "C4", "D4", "E4", "F4"),
           sep = "~", fill = "right")

# Rename column names
colnames(data) <- colnames(col_names)

In [20]:
%%R
dim(Raw)

[1] 34710    22


In [21]:
%%R
# List of all the files in teh dataset folder
files <- list.files(path = "dataset", full.names = T)
# delete all the files to keep the folder ready for next job
# unlink(files) - Keep in view for v3
# Write used raw data into [Used] folder to keep track of jobs
write.csv(Raw, paste0("Used/Raw_", Sys.Date(), ".csv"), row.names = F)


In [22]:
%%R
# Fist report data selection
select_data <- data %>%
  select(`SURNAME-PAYER`,
         `FIRST-NAMES-PAYER`,
         `PAYER-PHONE-CELL`,
         `PAYER-PHONE-WORK`,
         `PAYER-PHONE-HOME`,
         `AGENT-ALIAS-NAME`,
         `CONTRACT-PAYMENT-STATUS-REASON`,
         `CONTRACT-PAYMENT-FREQUENCY`,
         VALUE2,
         `NUMBER-OF-OUTSTANDING-PAYMENTS`,
         `CONTRACT-ID`) %>%
  filter(`CONTRACT-PAYMENT-STATUS-REASON` %in% c("ACTIVE", "PAID-UP", "PAYMENT-LAPSED")) %>%
  filter(`NUMBER-OF-OUTSTANDING-PAYMENTS` > 1)

# Second report data selection
next_select_data <- data %>%
  select(`SURNAME-PAYER`,
         `FIRST-NAMES-PAYER`,
         `PAYER-PHONE-CELL`,
         `PAYER-PHONE-WORK`,
         `PAYER-PHONE-HOME`,
         `AGENT-ALIAS-NAME`,
         `CONTRACT-PAYMENT-STATUS-REASON`,
         `CONTRACT-PAYMENT-FREQUENCY`,
         VALUE2,
         `NUMBER-OF-OUTSTANDING-PAYMENTS`,
         `CONTRACT-ID`,
         `PREVIOUS-CONTRACT-PAYMENT-POSITION`) %>%
  filter(`CONTRACT-PAYMENT-STATUS-REASON` %in% c("ACTIVE", "PAID-UP", "PAYMENT-LAPSED")) %>%
  filter(`NUMBER-OF-OUTSTANDING-PAYMENTS` > 1) %>%
  filter(`PREVIOUS-CONTRACT-PAYMENT-POSITION` < 0)

In [23]:
%%R
# Write and save both files
cleaned_alpha <- write.xlsx(select_data, paste0("Cleaned/Cleaned_alpha_", Sys.Date(), ".xlsx"), row.names = F)
cleaned_beta <- write.xlsx(next_select_data, paste0("Cleaned/Cleaned_beta_", Sys.Date(), ".xlsx"), row.names = F)